<a href="https://colab.research.google.com/github/faisalnawazmir/Econometrics-ML_for_Finance/blob/main/ayesha__Fama_q5_Factor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fama French Q5 Factor Model

In [40]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm 
from tabulate import tabulate
import time
np.random.seed(12345)

In [41]:
#copying data from excel file 25 portfoli
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
file = r'/gdrive/My Drive/data/q_q5_data_sheet.xlsx'
df = pd.read_excel(file, sheet_name='Sheet1')
df.head()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


,Date,MKt,Me,ROE,I/A,Reg,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25
0,2000-01-01,-0.040109,-0.009489,0.003310,0.015903,-0.040229,0.178104,0.149602,0.142498,0.121144,0.110616,0.117735,0.110664,0.092240,0.068990,0.076588,0.064833,0.058543,0.049305,0.035943,0.026366,0.031131,0.017331,0.001694,-0.009736,-0.014255,-0.015920,-0.021284,-0.008742,-0.037421,-0.035024
1,2000-01-02,-0.035767,-0.005061,0.008605,0.018628,-0.016751,0.092635,0.107413,0.108822,0.108591,0.111967,0.111211,0.119137,0.116468,0.120271,0.116661,0.101254,0.098793,0.109843,0.072183,0.070388,0.074738,0.064502,0.077777,0.083013,0.083013,0.104342,0.075977,0.085660,0.080385,0.090332
2,2000-01-03,-0.081055,0.013123,0.001593,0.007759,0.187280,0.132933,0.111175,0.102088,0.092935,0.091619,0.087737,0.089550,0.080550,0.045984,0.025300,0.020605,0.034820,0.055351,0.061460,0.068001,0.070566,0.112957,0.103677,0.108457,0.120620,0.116043,0.125302,0.154267,0.093937,0.108526
3,2000-01-04,0.022559,0.019400,0.016058,0.022607,-0.003890,0.019185,0.023567,0.046560,0.033526,0.054471,0.060460,0.068659,0.065226,0.065841,0.024099,0.018128,0.013584,0.007942,0.003133,-0.027661,-0.019598,-0.010939,0.000233,-0.005809,-0.002867,0.013325,0.008685,0.025226,0.049480,0.052285
4,2000-01-05,-0.001434,0.011741,0.023866,0.014866,0.046271,0.004970,-0.007078,-0.009864,-0.000072,0.003017,0.014748,0.014504,0.016163,0.008998,0.035545,0.028835,0.036682,0.041242,0.036976,0.053052,0.062109,0.054398,0.057881,0.079518,0.072934,0.087000,0.090995,0.122150,0.112607,0.164821


In [42]:
#N = 25  # of firms in an economy, here # of portfolios, here len(data)
T = len(df)  # length of periods # here 1136#df.columns.array
#  data
data = df.loc[:,'p1':'p25']
#data.shape#(T,N)
#x,y=data.shape
#N=y#  # of firms in an economy, here # of portfolios, here len(data)
#T = data.shape[0]
N = data.shape[1]

In [45]:
# simulate factor loadings (ie, coefficients)
b_mkt = np.zeros((1,N)) # true loadings for market factor
b_me = np.zeros((1,N)) # true loadings for size factor
b_roe = np.zeros((1,N)) # true loadings for value factor
b_ia = np.zeros((1,N)) # true loadings for value factor
b_reg = np.zeros((1,N)) # true loadings for value factor

In [46]:
#  factor returns
mkt = df['MKt'] # market factor
me = df['Me'] # size factor
roe = df['ROE'] # value factor
ia = df['I/A'] # market factor
reg=df['Reg']# umd is growth factor


In [47]:
#data.iloc[:,1]
fctrs = df[["MKt","Me","ROE","I/A","Reg"]]
fctrs.head()


,MKt,Me,ROE,I/A,Reg
0,-0.040109,-0.009489,0.003310,0.015903,-0.040229
1,-0.035767,-0.005061,0.008605,0.018628,-0.016751
2,-0.081055,0.013123,0.001593,0.007759,0.187280
3,0.022559,0.019400,0.016058,0.022607,-0.003890
4,-0.001434,0.011741,0.023866,0.014866,0.046271


In [48]:
no_factors=fctrs.shape[1]

In [49]:
# Step 1: time series regression firm by firm
#fctrs = np.c_[mkt, smb, hml]
# of factors
b = np.zeros(no_factors)
for i in range(N): # firm-by-firm time series regression
  model = LinearRegression()
  model.fit(fctrs,data.iloc[:,i])
  b = np.c_[b,model.coef_]

b  = np.transpose(b[:,1:])
b.shape
#b should be 0.1, 0.2

(25, 5)

In [50]:
#b is bhat
#b
#b.shape# (N,#factors)
df_bhat = pd.DataFrame(b)
df_bhat.columns = ['b_01','b_02','b_03','b_04','b_05']# change according factos
#print(df_bhat.head())

from google.colab import  drive
df_bhat.to_csv('/gdrive/My Drive/data/bhat_ayesha_q5_factors.csv')

In [51]:
# Step 2: cross sectional regression time-by-time
f = np.zeros(no_factors)
for i in range(T): # time-by-time cross sectional regression
  model = LinearRegression()
  model.fit(b,data.iloc[i,:])
  f = np.c_[f,model.coef_]

f = np.transpose(f[:,1:])
df_gamma = pd.DataFrame(f)
df_gamma.columns = ['mkt','me','roe','ia','reg']
print(df_gamma.describe())
print('Median estimate:', np.median(f,axis=0))
print('Standard errors (without Shanken correction):', np.std(f,axis=0)/T**.5)

              mkt          me         roe          ia         reg
count  240.000000  240.000000  240.000000  240.000000  240.000000
mean     0.015018    0.005738   -0.006092   -0.010960   -0.017151
std      0.384679    0.086180    0.138567    0.172002    0.262379
min     -1.420815   -0.365300   -0.425050   -0.532216   -0.917219
25%     -0.271659   -0.047970   -0.091730   -0.114871   -0.169259
50%      0.024505    0.009631   -0.001357   -0.015663    0.012354
75%      0.298281    0.058582    0.082053    0.094900    0.149685
max      1.006490    0.261185    0.401781    0.692027    1.406447
Median estimate: [ 0.02450481  0.00963138 -0.00135716 -0.01566268  0.01235392]
Standard errors (without Shanken correction): [0.02477914 0.00555132 0.00892583 0.01107952 0.01690119]


In [52]:
mu=np.median(f,axis=0)
sigma=(np.std(f,axis=0))/T**.5
tstat=mu/sigma
tstat
#tstat<1.96

array([ 0.98892895,  1.73497022, -0.15204892, -1.41366124,  0.73094962])

In [53]:
f.shape # (T,#factors)# f or df I assume is gamma hat
df_gamma.to_csv('/gdrive/My Drive/data/ayesha_gamma_q5_factors.csv')

In [39]:
#df.head()#f and df are same, gamma hat

In [54]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms

In [55]:
data_a=data.to_numpy()
fctrs_a=fctrs.to_numpy()

In [56]:
# Compute standard error with bootstrapping (Shanken correction unnecessary)
# One simple bootstrapping 

indices = np.random.randint(low=0,high=T, size=T)
new_data = data_a[indices,:]
new_fctrs = fctrs_a[indices,:]
xx = indices[0]
print(xx)
print(new_data[0,:])
print(data_a[xx,:])


226
[-0.06559197 -0.05499236 -0.03728318 -0.03858488 -0.02245933 -0.00375666
 -0.00410523 -0.00632384 -0.00083551  0.01072545  0.023182    0.01169267
  0.01095736  0.00725931  0.00893491  0.00212043  0.01602235  0.00517638
 -0.0077149   0.00873489 -0.00284707  0.00430922  0.0169501   0.01647544
  0.02539289]
[-0.06559197 -0.05499236 -0.03728318 -0.03858488 -0.02245933 -0.00375666
 -0.00410523 -0.00632384 -0.00083551  0.01072545  0.023182    0.01169267
  0.01095736  0.00725931  0.00893491  0.00212043  0.01602235  0.00517638
 -0.0077149   0.00873489 -0.00284707  0.00430922  0.0169501   0.01647544
  0.02539289]


In [57]:
def efhats(data,fctrs): # function to compute average estimated factor return
  T = data.shape[0]
  N = data.shape[1]
  # Step 1: Time series regression
  x = np.c_[np.ones(T),fctrs]
  M = np.linalg.inv(x.T @ x) @ x.T
  bhat = (M @ data)[1:,:].T

  # Step 2: cross sectional regression
  # Include other control variables here (eg, firm characteristics)
  x = np.c_[np.ones(N),bhat]
  M = np.linalg.inv(x.T @ x) @ x.T
  fhat = (M @ data.T)[1:,:].T
  df = pd.DataFrame(fhat)
  df.columns = ['mkt','me','roe','ia','reg']
  E_fhat = np.average(fhat,axis=0)
  return E_fhat 


In [58]:
E_fhat = efhats(data_a,fctrs_a)
simn = 10**3 # of bootstrapping
t = time.time() # bootstrapping start time 
for i in range(simn): 
  indices = np.random.randint(low=0,high=T,size=T) # resample by rows
  new_data = data_a[indices,:] # resampled stock return data
  new_fctrs = fctrs_a[indices,:] # resampled factor return data
  E_fhat = np.c_[E_fhat,efhats(new_data,new_fctrs)]
  # to check bootstrapping status
  if i % 100 == 0: print(i)

print('Simulation duration:', time.time()-t, 'seconds.')
df = pd.DataFrame(E_fhat.T)
df.columns = ['mkt','me','roe','ia','reg']
print(df.describe())

0
100
200
300
400
500
600
700
800
900
Simulation duration: 0.9569318294525146 seconds.
               mkt           me          roe           ia          reg
count  1001.000000  1001.000000  1001.000000  1001.000000  1001.000000
mean      0.002550     0.001385     0.002522     0.000103    -0.001348
std       0.023023     0.005433     0.007975     0.009651     0.014863
min      -0.107956    -0.017547    -0.021514    -0.034349    -0.047553
25%      -0.010580    -0.002069    -0.002793    -0.006007    -0.010709
50%       0.002550     0.001466     0.002168    -0.000117    -0.001471
75%       0.017345     0.004769     0.007615     0.005860     0.007676
max       0.080747     0.022525     0.031173     0.043673     0.059049


In [59]:
print('FM regression esimate of expeced factor returns: \n', np.average(E_fhat.T,axis=0))
print('FM regression esimate of standard errors: \n', np.std(E_fhat.T,axis=0))

FM regression esimate of expeced factor returns: 
 [ 0.00255043  0.00138546  0.00252218  0.00010299 -0.00134818]
FM regression esimate of standard errors: 
 [0.02301129 0.00543016 0.0079706  0.0096462  0.01485591]


In [60]:
#bootstrap t stats
mu_boot=np.average(E_fhat.T,axis=0)
sigma_boot=np.std(E_fhat.T,axis=0)
tstat_boot=mu_boot/sigma_boot
tstat_boot

array([ 0.11083371,  0.25514099,  0.31643572,  0.01067681, -0.09075044])

https://quant.stackexchange.com/questions/35781/how-to-interpret-the-grs-f-test-values
#grs test
https://stackoverflow.com/questions/24074481/fama-macbeth-regression-in-python-pandas-or-statsmodels

In [ ]:
pip install rpy2

In [61]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [62]:
%%R -i df -w 5 -h 5 --units in -r 200
# import df from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution

install.packages("GRS.test", repos='http://cran.us.r-project.org', quiet=TRUE)
library(GRS.test)


In [63]:
%%R -i data,fctrs

GRS.MLtest(data, fctrs)

$GRS.stat
          GRS
[1,] 1.146032

$GRS.pval
           GRS
[1,] 0.2943245

$thetas
         [,1]
[1,] 1.056589

$theta
          [,1]
[1,] 0.9316159

$ratio
          [,1]
[1,] 0.8817203



In [64]:
%%R -o model -o coef
model<-GRS.MLtest(data, fctrs)
coef<-model$GRS.stat

In [65]:
print(model)

$GRS.stat
          GRS
[1,] 1.146032

$GRS.pval
           GRS
[1,] 0.2943245

$thetas
         [,1]
[1,] 1.056589

$theta
          [,1]
[1,] 0.9316159

$ratio
          [,1]
[1,] 0.8817203




W statistic given in (7) of GRS (1989) <DOI:10.2307/1913625>

GRS.stat=GRS test statistic

GRS.pval=its p-value

theta=maximum Sharpe ratio of the K factor portfolios

thetas=slope of the efficient frontier based on all assets

ratio=theta/thetas, proportion of the potential efficiency

References
Gibbons, Ross, Shanken, 1989. A test of the efficiency of a given portfolio, Econometrica, 57,1121-
1152. <DOI:10.2307/1913625>


https://towardsdatascience.com/guide-to-r-and-python-in-a-single-jupyter-notebook-ff12532eb3ba

https://www.linkedin.com/pulse/interfacing-r-from-python-3-jupyter-notebook-jared-stufft/
how to switch R and python


In [66]:
print(coef)

[[1.14603183]]
